In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import os

import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import  roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Procesamiento 
#variable de interes
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

# Se combina el título de la película con la trama
dataTraining_2 = dataTraining.copy()
dataTraining_2['title_plot'] = dataTraining_2['title'] + ' - ' + dataTraining_2['plot']
dataTraining_2.drop(columns=['title','plot','rating'], inplace=True)

In [5]:
#Encoder
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub
import seaborn as sns

# Importación el módulo TF Hub del Universal Sentence Encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
embed = hub.Module(module_url)

# Codificación de las frases anteriormente definidas con la libreria tensorflow
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings = session.run(embed(dataTraining_2['title_plot']))

#df con encoder y el indice de fila
x_embed = pd.DataFrame(sentences_embeddings)
x_embed.index = dataTraining_2.index

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [6]:
# PostProcesamiento
#Se adiciona el año de la película a la matriz de embedding
dataTraining_3 = pd.concat([dataTraining_2, x_embed], axis=1)
dataTraining_3.drop(columns=['title_plot', 'genres'], inplace=True)

# Se estandariza la data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataTraining_3 = scaler.fit_transform(dataTraining_3)

In [7]:
# Se separa la data en conjunto de entrenamiento y conjunto de validación
X_train, X_test, y_train_genres, y_test_genres = train_test_split(dataTraining_3, y_genres, test_size=0.33, random_state=42)

In [8]:
# Importación de librerías para la implementación de la red neuronal

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.preprocessing import sequence
from livelossplot import PlotLossesKeras
from keras.callbacks import EarlyStopping
%matplotlib inline

dims = dataTraining_3.shape[1]
var_out = y_genres.shape[1]

K.clear_session()
model = Sequential()  
model.add(Dense(512, input_shape=(dims,), activation='sigmoid'))
model.add(Dense(var_out, activation='sigmoid'))

# Definición de función de perdida con parámetros definidos en la función nn_model_params
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

# Definición de la función EarlyStopping con parámetro definido en la función nn_model_params
early_stopping = EarlyStopping(monitor="val_loss", patience = 10)

# Entrenamiento de la red neuronal con parámetros definidos en la función nn_model_params
model.fit(X_train, y_train_genres, 
          validation_data = (X_test, y_test_genres),
          epochs=6,
          batch_size=64,
          callbacks=[early_stopping],
          verbose=0 
          )

In [11]:
import joblib
# Exportar modelo a archivo binario .pkl
joblib.dump(scaler, 'proy_final/scaler.pkl', compress=3)

FileNotFoundError: [Errno 2] No such file or directory: 'proy_final/scaler.pkl'

In [ ]:
# serializar el modelo a JSON
model_json = model.to_json()
with open("proy_final/model.json", "w") as json_file:
    json_file.write(model_json)
# serializar los pesos a HDF5
model.save_weights("proy_final/model.h5")
print("Modelo Guardado!")

In [ ]:
from keras.models import model_from_json
# cargar json y crear el modelo
json_file = open('proy_final/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# cargar pesos al nuevo modelo
loaded_model.load_weights("proy_final/model.h5")
print("Cargado modelo desde disco.")


In [ ]:
from tensorflow.keras.models import load_model

# evaluate loaded model on test data
loaded_model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])


In [ ]:
# Predicción del modelo de clasificación
y_pred_genres = loaded_model.predict(X_test)


In [ ]:

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

In [ ]:
from proy_final.proyecto_deployment import transformar

In [ ]:
df_datos = pd.DataFrame({'year': 1999, 'title': 'Message in a Bottle', 'plot': "who meets by fate ,  shall be sealed by fate .  theresa osborne is running along the beach when she stumbles upon a bottle washed up on the shore .  inside is a message ,  reading the letter she feels so moved and yet she felt as if she has violated someone ' s thoughts .  in love with a man she has never met ,  theresa tracks down the author of the letter to a small town in wilmington ,  two lovers with crossed paths .  but yet one can ' t let go of their past ."
                        }, index=[0])
df_datos

a = transformar(df_datos)
a

### Tratamiento para nuevos datos

In [ ]:
# Para predecir en la base de datos de test de kaggle hay que hacer el mismo tratamiento:

dataTesting_2 = dataTesting.copy()
dataTesting_2['title_plot'] = dataTesting_2['title'] + ' - ' + dataTesting_2['plot']
dataTesting_2.drop(columns=['title','plot'], inplace=True)

tf.disable_eager_execution()

# Importación el módulo TF Hub del Universal Sentence Encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
embed = hub.Module(module_url)

# Codificación de las frases anteriormente definidas con la libreria tensorflow
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings_2 = session.run(embed(dataTesting_2['title_plot']))

x_test_embed = pd.DataFrame(sentences_embeddings_2)
x_test_embed.index = dataTesting_2.index

dataTesting_3 = pd.concat([dataTesting_2, x_test_embed], axis=1)
dataTesting_3.drop(columns=['title_plot'], inplace=True)
dataTesting_3 = scaler.transform(dataTesting_3)
dataTesting_3

### Predicción nuevos datos

In [ ]:
y_pred_genres_2 = model.predict(dataTesting_3)

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

res = pd.DataFrame(y_pred_genres_2, index=dataTesting.index, columns=cols)
res.to_csv('pred_calib_param_3.csv', index_label='ID')
res.head()

### API

In [ ]:
model

In [ ]:
# Nota: lo entreno otra vez uno diferente, para validar que la infraestructura del anterior no incide en el problema de guardado

In [ ]:
K.clear_session()

# Definición red neuronal con la función Sequential()
model2 = Sequential()
    
# Definición de las capas de la red con el número de neuronas y la función de activación definidos en la función nn_model_params
model2.add(Dense(512, input_shape=(dims,), activation='sigmoid'))
model2.add(Dense(var_out, activation='sigmoid'))

# Definición de función de perdida con parámetros definidos en la función nn_model_params
model2.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])

# Definición de la función EarlyStopping con parámetro definido en la función nn_model_params
early_stopping = EarlyStopping(monitor="val_loss", patience = 10)

# Entrenamiento de la red neuronal con parámetros definidos en la función nn_model_params
model2.fit(X_train, y_train_genres,
          validation_data = (X_test, y_test_genres),
          epochs=20,
          batch_size=64,
          callbacks=[early_stopping, PlotLossesKeras()],
          verbose=True
          )

In [ ]:
import joblib
import tensorflow as tf

# guardar el modelo
#joblib.dump(model, 'API/modelo_red_neuronal.pkl', compress=3)
joblib.dump(scaler, 'model_deploy/scaler.pkl', compress=3)

#tf.saved_model.save(model, 'API/modelo_red_neuronal3')

model2.save('model_deploy/modelo_red_neuronal_h5.h5')

In [ ]:
from tensorflow.keras.models import load_model

modelo_cargado = load_model('API/modelo_red_neuronal_h5.h5')

In [ ]:
modelo_cargado.predict(X_test)

In [ ]:
import pandas as pd

In [ ]:
dataTesting.loc[1]['plot']

In [ ]:
df_datos = pd.DataFrame({'year': 1999, 'title': 'Message in a Bottle', 'plot': "who meets by fate ,  shall be sealed by fate .  theresa osborne is running along the beach when she stumbles upon a bottle washed up on the shore .  inside is a message ,  reading the letter she feels so moved and yet she felt as if she has violated someone ' s thoughts .  in love with a man she has never met ,  theresa tracks down the author of the letter to a small town in wilmington ,  two lovers with crossed paths .  but yet one can ' t let go of their past ."
                        }, index=[0])
df_datos

In [ ]:
from API.proyecto2_deployment import transformar

In [ ]:
a = transformar(df_datos)
a

In [ ]:
from flask import Flask
from flask_restx import Api, Resource, fields

In [ ]:

app = Flask(__name__)

api = Api(
    app, 
    version='1.0', 
    title='Clasificación de género de películas',
    description='Clasificación de género de películas')

ns = api.namespace('predict', 
     description='Predicción géneros de la película')

parser = api.parser()

parser.add_argument(
    'year', 
    type=int, 
    required=True, 
    help='Año del lanzamiento de la película', 
    location='args')

parser.add_argument(
    'title', 
    type=str, 
    required=True, 
    help='Nombre de la película', 
    location='args')

parser.add_argument(
    'plot', 
    type=str, 
    required=True, 
    help='Trama de la película', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [ ]:
@ns.route('/')
class PrediccionApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": transformar(args)  
        }, 200

In [ ]:
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=8888)

In [ ]:
!pip freeze '/requirem.txt'